Che tipo hanno le seguenti espressioni? Cercare di inferirlo a mente (o su carta) e poi controllare se la risposta è corretta facendo valutare l'espressione all'interprete:

1. `fun x -> fun y -> x y y`
2. `(fun x -> 2. , fun x -> x 2.)`
3. `fun x -> fun y -> if x=true then y 2 else y 1`
4. `(fun x -> fun y -> if x=true then y 2 else y 1)(false)(fun x -> x+1)`

**Esercizio 2.10.** Scrivere una funzione (o meglio, un predicato) `set` che prende una lista e verifica (restituendo `true` o `false`) se i suoi elementi sono tutti diversi tra loro, cioè se la lista è una corretta rappresentazione di un insieme. La funzione deve avere tipo `'a -> bool`, in modo da poter essere essere applicata ad insiemi con elementi di qualunque tipo.

In [6]:
(* Questa soluzione ordina la lista, poi controlla che non ci siano due elementi uguali consecutivi
 * Questa soluzione è più efficiente (O(n * log n), il costo dell'ordinamento) ma richiede che sia
 * possibile ordinare gli elementi della lista (non è sempre vero, dipende dal tipo).
 *)
let set1 lista =
  let rec no_dupl hd tl = match tl with
    | [] -> true
    | x::xs -> if hd = x then false else no_dupl x xs
    in
  match List.stable_sort compare lista with
    | [] -> true
    | hd::tl -> no_dupl hd tl;;

(* Questa soluzione cerca, per ogni elemento della lista, se è presente nella coda della lista.
 * Basta controllare la coda perché, se fosse presente prima, sarebbe esso stesso nella coda
 * dell'elemento uguale precedente.
 * Questa soluzione è meno efficiente (O(n^2)) ma richiede solo di poter controllare l'uguaglianza
 * di due elementi della lista (che è un requisito più debole dell'ordinamento).
 *)
let rec set2 lista =
  match lista with
    | [] -> true
    | x::xs -> if List.exists (fun y -> y = x) xs then false else set2 xs;;


set1 [1; 2; 3];;
set1 [1; 2; 1];;

set2 [1; 2; 3];;
set2 [1; 2; 1];;

val set1 : 'a list -> bool = <fun>


val set2 : 'a list -> bool = <fun>


- : bool = true


- : bool = false


- : bool = true


- : bool = false


**Esercizio 2.11.** Un multi-insieme (o multiset) estende il concetto di insieme consentendo la possibilità di avere più occorrenze dello stesso elemento. Un multiset può essere definito come una lista di coppie `(x,n)` in cui `x` è l'elemento considerato e `n` è il numero di occorrenze di quell'elemento. Scrivere una funzione `multiset` che, data una lista di coppie di tipo `'a*int`, verifica se è una corretta rappresentazione di un multiset (contentente coppie che sono tutte diverse tra loro nel primo elemento). 

In [10]:
let fst (a, b) = a;;

(* multiset prende in input una lista di coppie, e deve verificare se la lista formata dai primi
 * elementi di ogni coppia nella lista di partenza è un set.
 * Questa prima implementazione usa una funzione ausiliaria per costruire questa seconda lista,
 * su cui poi applica la funzione set dell'esercizio precedente.
 * La scelta di set1 piuttosto che set2 è arbitraria, e porta pro e contro descritti precedentemente. 
 *)
let multiset1 bag =
  let rec prima_coppia cose =
    match cose with
      | [] -> []
      | x::xs -> fst x::prima_coppia xs
    in
  set1 (prima_coppia bag);;

(* prima_coppia sta applicando la funzione fst ad ogni elemento della lista. Questo pattern è
 * esattamente una List.map, quindi usiamo questa funzione higher-order per definire prima_coppia.
 *)
let multiset2 bag =
  let prima_coppia cose = List.map fst cose in
  set1 (prima_coppia bag);;

(* o più semplicemente *)
let multiset3 bag = set1 (List.map fst bag);;

multiset1 [('a', 10); ('b', 3)];;
multiset1 [('a', 10); ('a', 3)];;

multiset2 [('a', 10); ('b', 3)];;
multiset2 [('a', 10); ('a', 3)];;

val fst : 'a * 'b -> 'a = <fun>


val multiset1 : ('a * 'b) list -> bool = <fun>


- : bool = true


- : bool = false


**Esercizio 2.1.** Scrivere una funzione `genera_lista` che prende un intero positivo `n` e restituisce una lista contenente la lista `[1; 2; ... n]`. Nel caso in cui `n` sia minore o uguale di zero restituisce la lista vuota.

In [16]:
let genera_lista1 n =
  let rec aux n =
    match n with
      | 0 -> []
      | x -> x::aux (x - 1)
    in
  List.rev (aux n);;

let rec genera_lista2 n =
  match n with
    | 0 -> []
    | n -> genera_lista2 (n-1) @ [n];;

let genera_lista3 n =
  let rec counter n c = if c <= n then c::counter n (c+1) else [] in
  counter n 1;;

genera_lista1 4;;
genera_lista2 4;;
genera_lista3 4;;

val genera_lista1 : int -> int list = <fun>


val genera_lista2 : int -> int list = <fun>


val genera_lista3 : int -> int list = <fun>


- : int list = [1; 2; 3; 4]


- : int list = [1; 2; 3; 4]


- : int list = [1; 2; 3; 4]


**Esercizio 2.2.** Scrivere una funzione `media` che prende una lista di interi e calcola la media dei suoi elementi. Farlo senza ricorsione esplicita, ma usando funzioni higher-order.

In [29]:
List.fold_left;;
List.fold_right;;

- : ('a -> 'b -> 'a) -> 'a -> 'b list -> 'a = <fun>


- : ('a -> 'b -> 'b) -> 'a list -> 'b -> 'b = <fun>


In [31]:
(* Semplice utility per calcolare il rapporto di una coppia di interi (la usiamo più volte sotto) *)
let rapporto (sum, len) =  float_of_int sum /. float_of_int len;;

(* Definiamo la media come la somma diviso la lunghezza della lista. Per calcolare
 * la somma, usiamo un fold.
 *)
let media_sum lista =
  let sum list = List.fold_right (+) list 0 in
  rapporto (sum lista, List.length lista);;

(* Somma si può definire anche con fold_left *)
let sum list = List.fold_left (+) 0 list;;


(* Implementazione ricorsiva con una funzione ausiliaria che calcola sum e length scorrendo
 * la lista una sola volta. Messa qui per il confronto con la versione higher-order.
 *)
let media_aux_rec lista =
  let rec aux (sum, len) list =
    match list with
      | [] -> (sum, len)
      | x::xs -> aux (sum + x, len + 1) xs
    in
  rapporto (aux (0, 0) lista);;

(* Concettualmente stessa implementazione di prima, ma aux è definita usando fold invece che
 * ricorsione esplicita
 *)
let media_aux list =
  let aux list =
    List.fold_left (fun (sum, len) elem -> (sum + elem, len + 1)) (0, 0) list in
  rapporto (aux list);;

(* o più semplicemente *)
let media_aux2 list =
  rapporto (List.fold_left (fun (sum, len) elem -> (sum + elem, len + 1)) (0, 0) list);;

media_sum [1; 2; 10];;

val rapporto : int * int -> float = <fun>


val media_sum : int list -> float = <fun>


val sum : int list -> int = <fun>


val media_aux_rec : int list -> float = <fun>


val media_aux : int list -> float = <fun>


val media_aux2 : int list -> float = <fun>


- : float = 4.33333333333333304


Che tipo ha l'espressione `List.fold_left (+) 0`?

**Esercizio bonus**

Date le due definizioni
```ocaml
let sum lista = List.fold_left (+) 0 lista;;
let sum = List.fold_left (+) 0;;
```
la prima prende un parametro (quindi è una funzione), mentre la seconda no. Tuttavia, sono due definizioni equivalenti. Com'è possibile?

*Hint*: quanti argomenti prende `List.fold_left`?